In [3]:
!nvidia-smi

Sun Nov 24 08:07:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0              27W /  70W |    575MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install pypdf

In [5]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -U langchain-community

In [6]:
!pip install sentence_transformers

In [7]:
!pip install llama-index==0.9.39

In [8]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [9]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [10]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [11]:
documents

[Document(id_='3779f154-a1c1-43d9-8dca-c2d0826b03e7', embedding=None, metadata={'page_label': '1', 'file_name': 'ULM.pdf', 'file_path': '/content/Data/ULM.pdf', 'file_type': 'application/pdf', 'file_size': 421696, 'creation_date': '2024-11-24', 'last_modified_date': '2024-11-24', 'last_accessed_date': '2024-11-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Uniﬁed Language Model Pre-training for\nNatural Language Understanding and Generation\nLi Dong∗ Nan Yang∗ Wenhui Wang∗ Furu Wei∗† Xiaodong Liu Yu Wang\nJianfeng Gao Ming Zhou Hsiao-Wuen Hon\nMicrosoft Research\n{lidong1,nanya,wenwan,fuwei}@microsoft.com\n{xiaodl,yuwan,jfgao,mingzhou,hon}@microsoft.com\nAbstract\nThis paper presents a new UNIﬁed pre-trained Language Model (UNILM) that\ncan be ﬁne

In [12]:
#Create a prompt
system_prompt = """You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided."""


# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [13]:
!huggingface-cli login #make sure the the token should have read access


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read)

In [14]:
import torch

llm = HuggingFaceLLM(
    context_window=4096, #input size of the llm - 4096 token as input
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [16]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [17]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [22]:
#from llama_index.llms import ServiceContext
#from llama_index.core import ServiceContext
query_engine = index.as_query_engine()
response = query_engine.query("What is Deep Learning?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [23]:
print(response)

Deep learning is a subset of machine learning that uses artificial neural networks to model and solve complex problems. It is inspired by the structure and function of the human brain and is designed to learn from data, identify patterns, and make predictions or decisions without being explicitly programmed. Deep learning has been used in a variety of applications, including image and speech recognition, natural language processing, computer vision, and autonomous systems.
